In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda_v2 = ccl.Cosmology(
    Omega_c=0.94, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_w0wa = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0=-0.8, wa=-1.0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)


In [ ]:
def z_to_a(z: np.ndarray) -> np.ndarray:
    return 1/(1+z)

def a_to_z(a: np.ndarray) -> np.ndarray:
    return 1/(1-a)

def int_j(cosmo, z_vals):
    a_vals = z_to_a(z_vals)
    j = 1./cosmo.h_over_h0(a_vals)
    z_widths = z_vals[1:] - z_vals[0:-1]    
    return np.cumsum(j)[1:] * z_widths

def chiRatio(cosmo, z_d: np.ndarray, z_s: np.ndarray) -> np.ndarray:
    a_d = z_to_a(z_d)
    a_s = z_to_a(z_s)
    chi_d = cosmo.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
    chi_s = cosmo.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
    rat = chi_s/chi_d
    return rat

def int_j_ratio(int_j_func, z_d: np.ndarray, z_s: np.ndarray) -> np.ndarray:
    int_j_s = int_j_func(z_s)
    int_j_d = int_j_func(z_d)
    return int_j_s/int_j_d

In [ ]:
n_obj = 1200
z_lens_mean = 0.74
z_lens_scale = 0.49
z_source_mean = 2.31
z_source_scale = 0.49

In [ ]:
z_lens_dist = scipy.stats.lognorm(scale=z_lens_mean, s=z_lens_scale)
z_source_dist = scipy.stats.lognorm(scale=z_source_mean, s=z_source_scale)

In [ ]:
z_lens_vals = z_lens_dist.rvs(n_obj)
z_source_vals = z_source_dist.rvs(n_obj)
mask = np.bitwise_and(z_source_vals - z_lens_vals > 0.1, z_source_vals<6)
z_lens_vals = z_lens_vals[mask]
z_source_vals = z_source_vals[mask]

In [ ]:
vals = chiRatio(cosmo, z_lens_vals, z_source_vals)
vals_no_lambda = chiRatio(cosmo_no_lambda, z_lens_vals, z_source_vals)
vals_wowa = chiRatio(cosmo_w0wa, z_lens_vals, z_source_vals)

err_level = 0.05
scatter = np.random.normal(0, err_level, size=len(z_source_vals))
errs = vals*err_level
errs_no_lambda = vals_no_lambda*err_level
errs_wowa = vals_wowa*err_level

In [ ]:
_ = plt.hist(z_lens_vals, bins=np.linspace(0, 6, 121), alpha=0.6, label="lens")
_ = plt.hist(z_source_vals, bins=np.linspace(0, 6, 121), alpha=0.6, label="source")
_ = plt.xlabel('z')
_ = plt.ylabel("Objects / [0.05]")

In [ ]:
zmin = 0.01
zmax = 6
nzBins = 300

In [ ]:
z_grid = np.linspace(zmin, zmax, nzBins)
z_widths = z_grid[1:] - z_grid[0:-1]
a_grid = z_to_a(z_grid)

In [ ]:
result1d = np.polynomial.Polynomial.fit(z_grid[1:], int_j(cosmo, z_grid), deg=6).convert()

In [ ]:
result1d.coef

In [ ]:
result1d

In [ ]:
int_j_func = np.polynomial.Polynomial(result1d.coef)

In [ ]:
check = int_j_ratio(int_j_func, z_lens_vals, z_source_vals)

In [ ]:
_ = plt.plot(z_grid[1:], int_j(cosmo, z_grid))
_ = plt.plot(z_grid, result1d(z_grid))
_ = plt.plot(z_grid, int_j_func(z_grid))

In [ ]:
_ = plt.plot(z_grid[1:], int_j(cosmo, z_grid) - result1d(z_grid[1:]))
_ = plt.plot(z_grid[1:], int_j(cosmo, z_grid) - int_j_func(z_grid[1:]))

In [ ]:
class PolyFit:

    def __init__(self, z_d, z_s, data, errors):
        self.z_d = z_d
        self.z_s = z_s
        self.data = data
        self.errors = errors

    @staticmethod
    def chi2Vals(data, errors, model):
        delta = (data - model)/errors
        return delta*delta
        
    @staticmethod
    def modelVals(z_d, z_s, params):
        poly = np.polynomial.Polynomial(params)
        int_j_s = poly(z_s)
        int_j_d = poly(z_d)
        return int_j_s/int_j_d

    @staticmethod
    def modelSpace(z_grid, params):        
        mesh = np.meshgrid(z_grid, z_grid)
        z_d = mesh[0]
        z_s = mesh[1]
        poly = np.polynomial.Polynomial(params)
        int_j_s = poly(z_s.ravel()).reshape(z_s.shape)
        int_j_d = poly(z_d.ravel()).reshape(z_d.shape)
        rat =  int_j_d/int_j_s
        return np.where(z_s-z_d > 0.1, rat, 0.)
    
    def model(self, params):
        return PolyFit.modelVals(self.z_d, self.z_s, params)

    def chi2(self, params):
        model = self.model(params)
        return PolyFit.chi2Vals(self.data, self.errors, model)
    
    def __call__(self, params):
        chi2v = self.chi2(params)
        return chi2v.sum()
    

In [ ]:
pf = PolyFit(z_lens_vals, z_source_vals, vals, errs)

In [ ]:
init_pars = result1d.coef

In [ ]:
result = scipy.optimize.minimize(pf, init_pars, method='nelder-mead', options={'xatol': 1e-8, 'disp': True, 'maxiter':2000})


In [ ]:
modeler = np.polynomial.Polynomial(result.x)

In [ ]:
_ = plt.plot(z_grid[1:], int_j(cosmo, z_grid))
_ = plt.plot(z_grid, 0.5*modeler(z_grid))

In [ ]:
modelSpace = PolyFit.modelSpace(z_grid, result.x)

In [ ]:
_ = plt.imshow(modelSpace, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{ds} = \chi_{s}/\chi_{d}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')